In [4]:
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AX4XfWgDRdlLwBgl1CO7Ti0-IDMBensMvDcxlQvTQWKEKfkqZVdDwz7FqrE



Successfully saved authorization token.


In [5]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name = name,
        overlay = True,
        control = True
    ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [14]:
# https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_DAILY
era5_2mt = (
    ee.ImageCollection('ECMWF/ERA5/DAILY')
    .select('mean_2m_air_temperature')
    .filter(ee.Filter.date('2019-07-01', '2019-07-31'))
)

vis2mt = {
  'min': 250,
  'max': 320,
  'palette': [
    '#000080', '#0000D9', '#4000FF', '#8000FF', '#0080FF', '#00FFFF', '#00FF80',
    '#80FF00', '#DAFF00', '#FFFF00', '#FFF500', '#FFDA00', '#FFB000', '#FFA400',
    '#FF4F00', '#FF2500', '#FF0A00', '#FF00FF'
  ]
}

my_map = folium.Map()
my_map.add_ee_layer(
    era5_2mt.reduce(ee.Reducer.mean()),
    vis2mt,
    '2 meter temperature'
)
display(my_map)

In [17]:
# https://developers.google.com/earth-engine/datasets/catalog/IDAHO_EPSCOR_GRIDMET
idaho_data = (
    ee.ImageCollection("IDAHO_EPSCOR/GRIDMET")
    .select('tmmn')
    .filter(ee.Filter.date('2019-07-01', '2020-07-01'))
)

vis = {
  'min': 250,
  'max': 320,
  'palette': [
    '#000080', '#0000D9', '#4000FF', '#8000FF', '#0080FF', '#00FFFF', '#00FF80',
    '#80FF00', '#DAFF00', '#FFFF00', '#FFF500', '#FFDA00', '#FFB000', '#FFA400',
    '#FF4F00', '#FF2500', '#FF0A00', '#FF00FF'
  ]
}

my_map = folium.Map()
my_map.add_ee_layer(
    idaho_data.reduce(ee.Reducer.mean()),
    vis,
    'idaho avg temp'
)

display(my_map)

In [20]:
# https://developers.google.com/earth-engine/datasets/catalog/IDAHO_EPSCOR_GRIDMET
idaho_data = (
    ee.ImageCollection("IDAHO_EPSCOR/GRIDMET")
    .select('pr')
    .filter(ee.Filter.date('2019-07-01', '2020-07-01'))
)

vis = {
  'min': 0,
  'max': 2000,
  'palette': [
    '#000080', '#0000D9', '#4000FF', '#8000FF', '#0080FF', '#00FFFF', '#00FF80',
    '#80FF00', '#DAFF00', '#FFFF00', '#FFF500', '#FFDA00', '#FFB000', '#FFA400',
    '#FF4F00', '#FF2500', '#FF0A00', '#FF00FF'
  ]
}

my_map = folium.Map()
my_map.add_ee_layer(
    idaho_data.reduce(ee.Reducer.sum()),
    vis,
    'idaho total precip'
)

display(my_map)